In [ ]:
# '/content/drive/MyDrive/FUNIX Progress/MLP303x_1.1-A_EN/data/amazon_baby_subset.csv'
# '/content/drive/MyDrive/FUNIX Progress/MLP303x_1.1-A_EN/data/important_words.json'

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import json

df = pd.read_csv('/content/drive/MyDrive/FUNIX Progress/MLP303x_1.1-A_EN/data/amazon_baby_subset.csv')
with open('/content/drive/MyDrive/FUNIX Progress/MLP303x_1.1-A_EN/data/important_words.json') as file:
  important_words = json.loads(file.read())

In [3]:
def remove_punctuation(text):
  import string
  return text.translate(string.punctuation)

df = df.fillna('')
df['review'].apply(remove_punctuation)

for word in important_words:
  df[word] = df['review'].apply(lambda s: s.split().count(word))

df.head()

,name,review,rating,sentiment,baby,one,great,love,use,would,like,easy,little,seat,old,well,get,also,really,son,time,bought,product,good,daughter,much,loves,stroller,put,months,car,still,back,used,recommend,first,even,perfect,nice,bag,...,looks,second,piece,box,pretty,trying,difficult,together,though,give,started,anything,last,company,come,returned,maybe,took,broke,makes,stay,instead,idea,head,said,less,went,working,high,unit,seems,picture,completely,wish,buying,babies,won,tub,almost,either
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [4]:
train, test = train_test_split(df, train_size = 0.8)
print(train.shape)
print(test.shape)

(42457, 197)
(10615, 197)


In [5]:
def extract_feature(df, feature_list, label_list):
  df['constant'] = 1

  feature_list = ['constant'] + feature_list

  return (df[feature_list].values, df[label_list].values)

In [6]:
train_feature, train_label = extract_feature(train, important_words, 'sentiment')
test_feature, test_label = extract_feature(test, important_words, 'sentiment')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
def score(feature, weight):
  return feature.dot(weight)

In [8]:
def proba(score):
  return 1 / (1 + np.exp(-score))

In [9]:
def feature_derivative(feature, error, weight, l2_penalty, not_constant):
  derivative = error.dot(feature)

  if(not_constant):
    derivative -= 2 * l2_penalty * weight

  return derivative

In [10]:
def log_likelihood(indicator, proba, score, l2_penalty, weight):
  return np.sum(indicator - 1)*score - np.log(1. + np.exp(-score) - l2_penalty*weight.dot(weight))

In [17]:
def gradient_descent(feature, label, weight, step_size, l2_penalty, max_iter):
  _indicator = (label == +1).astype(int)
  for iter in range(max_iter):
    _score = score(feature, weight)
    _proba = proba(_score)
    _error = _indicator - _proba
    
    for i in range(weight.shape[0]):
      weight[i] = weight[i] + step_size * feature_derivative(feature[:, i], _error, weight[i], l2_penalty, i)

  return weight

In [18]:
# run with L2 = 0
coefficients_0_penalty = gradient_descent(train_feature, train_label,
                                         weight=np.zeros((194,)),
                                         step_size=5e-6, l2_penalty=0, max_iter=501)

In [19]:
coefficients_0_penalty

array([-0.08931861,  0.09371216,  0.01397851,  0.75997745,  1.08556503,
        0.07118598, -0.29834402, -0.02245574,  0.98708227,  0.52505732,
       -0.08796177,  0.12413876,  0.47596689, -0.19408901,  0.15562583,
        0.00188947,  0.13553747, -0.14152587, -0.14130605, -0.36912065,
        0.14086336,  0.26980877, -0.03187947,  1.07239389, -0.05569999,
        0.00924617, -0.04971131,  0.14894137,  0.18412258, -0.19532545,
        0.11921168,  0.37765941, -0.05335496, -0.39805561,  0.71297907,
        0.45443945, -0.04589615, -0.1154675 ,  0.02139449, -0.10603539,
       -0.09708461,  0.0951874 ,  0.07177563,  0.07257429, -0.12173547,
        0.17051561, -0.00521381, -0.13741972,  0.30755235,  0.06917064,
       -0.32963147,  0.24942773,  0.40967295, -0.11842855, -0.10111426,
        0.1775308 ,  0.16480148, -0.11865885,  0.22825196,  0.12795065,
        0.02507701, -0.01308255,  0.14051887,  0.20923858,  0.14480247,
        0.11529175,  0.12128034, -0.38609943, -0.07196377, -0.16